<a href="https://colab.research.google.com/github/NeagrTavakol/Final_dataminning/blob/main/Copy_of_Final_projet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
#mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Read dev dataset**

In [131]:
dev_csv="/content/drive/MyDrive/Colab Notebooks/dev.csv"
dev_df = pd.read_csv(dev_csv,sep='\t')
dev_df=dev_df.drop("Unnamed: 0",axis=1)
dev_columns=dev_df.columns

**Read train dataset**

In [39]:
train_csv="/content/drive/MyDrive/Colab Notebooks/train.csv"
train_df = pd.read_csv(train_csv,sep='\t')
train_df=train_df.drop("Unnamed: 0",axis=1)
train_columns = train_df.columns

# **Tokenize Data**

**install packages**

In [126]:
# Install; note that the prefix "!" is not needed if you are running in a terminal
!pip install stanza


     |████████████████████████████████| 432 kB 5.6 MB/s 
     |████████████████████████████████| 174 kB 29.7 MB/s 
  Created wheel for emoji: filename=emoji-1.6.3-py3-none-any.whl size=170298 sha256=aae91f9ba9e9027390edcbf063d79d80bf848c01118f7b61ed1a3a6c7042e7c1
  Stored in directory: /root/.cache/pip/wheels/03/8b/d7/ad579fbef83c287215c0caab60fb0ae0f30c4d7ce5f580eade
Successfully built emoji


In [127]:
# Import the package
import stanza

In [128]:
# Download an Persiabn model into the default directory
print("Downloading Persian model...")
stanza.download('fa')

2022-01-28 12:36:24 INFO: Downloading default packages for language: fa (Persian)...


2022-01-28 12:36:32 INFO: Finished downloading models and saved to /root/stanza_resources.


**create models**

In [129]:
# Build a Persian pipeline, with all processors by default
print("Building a Persian pipeline...")
fa_nlp = stanza.Pipeline('fa')

2022-01-28 12:38:13 INFO: Loading these models for language: fa (Persian):
| Processor | Package |
-----------------------
| tokenize  | perdt   |
| mwt       | perdt   |
| pos       | perdt   |
| lemma     | perdt   |
| depparse  | perdt   |

2022-01-28 12:38:13 INFO: Use device: cpu
2022-01-28 12:38:13 INFO: Loading: tokenize
2022-01-28 12:38:13 INFO: Loading: mwt
2022-01-28 12:38:13 INFO: Loading: pos


Building a Persian pipeline...


2022-01-28 12:38:13 INFO: Loading: lemma
2022-01-28 12:38:13 INFO: Loading: depparse
2022-01-28 12:38:14 INFO: Done loading processors!


**find tokens of dataframe and save in array**

In [ ]:
# test_token_df['tokenized_sents'] = test_token_df.apply(lambda row: fa_nlp(row['comment']).sentences.words.text, axis=1)
tokenized_array=[]
for i in range(0,len(train_df)[:52465]):
  comment=train_df.iloc[i]["comment"]
  tokenized_comment = fa_nlp(comment)
  tokens_array=[]
  for i,sent in enumerate(tokenized_comment.sentences):
    for word in sent.words:
      tokens_array.append(word.text)
  tokenized_array.append(tokens_array)
#word.text, word.lemma, word.pos, word.head, word.deprel))

In [ ]:
#find tokens of rest of dataset (process interupted cause of volume of dataset)
# rest_train_df=train_df[52466:]
for i in range(52466,len(train_df)):
  comment=train_df.iloc[i]["comment"]
  tokenized_comment = fa_nlp(comment)
  tokens_array=[]
  for i,sent in enumerate(tokenized_comment.sentences):
    for word in sent.words:
      tokens_array.append(word.text)
  tokenized_array.append(tokens_array)

/usr/local/lib/python3.7/dist-packages/stanza/models/common/beam.py:86: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  prevK = bestScoresId // numWords


**save tokens array to csv file**

In [ ]:
#save tokens to array
import csv
with open('tokens_array.csv', 'w', newline='') as file:
    mywriter = csv.writer(file, delimiter=',')
    mywriter.writerows(tokenized_array)

**convert tokens array to dataframe **

In [ ]:
# convert tokenized_array to dataframe
tokenized_df=pd.DataFrame(tokenized_array)

**save dataframe to csv file**

In [ ]:
#save dataframe 
tokenized_df.to_csv('tokens_df.csv')

In [ ]:
# with open('tokens_array.csv', 'r', newline='') as file:
#   myreader = csv.reader(file, delimiter=',')
#   for rows in myreader:
#     print(rows)

## **Read tokenized dataet**

**read from array csv file **

In [ ]:
############

In [14]:
#read array csv file
tokens_array=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/tokens_array.csv",sep='\t',header=None)
# tokens_array.head()

## **Preproccessing**

**Remove punctuation**

In [15]:
from types import new_class
# from string import punctuation
import re

punctuation=['!','"','#','$','%','&',"'",'(',')','*','+','،','-','.',','"/",':',';','<','=','>','?','@','[',"\\","]","^",'_','`','{','|','}','~']
# tokens_array.to_numpy()
tokens_arr=[]
for i in tokens_array.to_numpy():
  sent=[]
  for j in i[0].split(','):
    if j not in punctuation:
      sent.append(j)
  tokens_arr.append(sent)

In [ ]:
for i in tokens_arr[:10]:
  print(i)

**Analyze Reviews Length**

In [16]:
import matplotlib.pyplot as plt
reviews_len = [len(x) for x in tokens_arr]
reviews_len=np.array(reviews_len)

In [ ]:
# pd.Series(tokens_arr).hist()
# plt.show()
# pd.Series(tokens_arr).describe()

In [ ]:
print("min:",reviews_len.min(),"max:",reviews_len.max(),"min:",reviews_len.mean(),"len:",len(reviews_len))

min: 3 max: 362 min: 19.003015873015872 len: 56700


In [108]:
# for i in range(6,29):
print(np.count_nonzero(reviews_len > 75))

689


**Remove outliers**

In [17]:
cleaned_rokens_arr=[x for x in tokens_arr if len(x)<75]
print(len(cleaned_rokens_arr))

55978


ثریا 
اینجا کد ورد امبدنیگ رو میتونی ببینی 
کلمات مختلفی هم باهاش امتحان کردم اوکی شد

## **Word embedding**

In [18]:
!pip install gensim
# !pip install nltk

In [19]:
#main_code
import gensim
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/Colab Notebooks/model.bin', binary=True)

In [ ]:
vector = model['خوب']
# see the shape of the vector (300,)
# vector.shape
vector

In [134]:
v=model['خوب']
v

array([ 0.358336, -0.115084,  0.146338, -0.257848, -0.008565, -0.091646,
        0.180381, -0.178729, -0.339027,  0.492287,  0.223761,  0.003133,
       -0.572335,  0.098732,  0.355269,  0.191529,  0.388377, -0.181705,
       -0.167962,  0.064537,  0.1801  , -0.087827,  0.219954, -0.350673,
       -0.136104, -0.392271,  0.10476 , -0.102721,  0.255268, -0.344888,
        0.118361,  0.007972, -0.208708,  0.377865, -0.34835 ,  0.311579,
        0.47357 ,  0.028622,  0.417335,  0.095775,  0.108944,  0.091   ,
       -0.20897 , -0.061072,  0.041395,  0.368861,  0.401004, -0.281428,
       -0.169198, -0.123401, -0.130192, -0.100334,  0.064119,  0.056904,
       -0.125578, -0.3283  ,  0.439989, -0.20006 , -0.075791, -0.359489,
       -0.22543 ,  0.33668 ,  0.194959,  0.070917, -0.182243,  0.220156,
       -0.026614, -0.151517,  0.166414,  0.420058,  0.477057, -0.084695,
        0.46363 , -0.148052, -0.519539, -0.097664,  0.469537, -0.011281,
       -0.044912, -0.196027,  0.183174,  0.12066 , 

In [133]:
model.most_similar('بهتر')

[('.بهتر', 0.8202100992202759),
 ('مناسبتر', 0.7822331190109253),
 ('مفیدتر', 0.7766575217247009),
 ('خوب', 0.7656928300857544),
 ('وبهتر', 0.7652808427810669),
 ('راحتر', 0.7449773550033569),
 ('راحتتر', 0.7429759502410889),
 ('..بهتر', 0.73868727684021),
 ('-بهتر', 0.7375583052635193),
 ('!بهتر', 0.7351512908935547)]

In [ ]:
model.most_similar('طعم')

**calculate vacors for each word**

In [138]:
vocabulary =model.wv.vocab

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


In [22]:
print(len(vocabulary),type(vocabulary))

966446 <class 'dict'>


In [23]:
vectors_arr=[]
for sent in cleaned_rokens_arr:
  vec_vocab=[model[word] for word in sent if word in vocabulary]# if model[word]
  vectors_arr.append(vec_vocab)

In [24]:
from sys import getsizeof
getsizeof(vectors_arr)

457360

In [ ]:
# vocabulary = word2vec.wv.vocab
# embedded_array=[]
# for sent in cleaned_rokens_arr:
#   vec_vocab=[word2vec.wv[word] for word in sent if word in vocabulary]
#   embedded_array.append(vec_vocab)

**save embedded words in csv file(too large!)>500mb**

In [ ]:
# #save vctors to array
# import csv
# with open('vectors_arr.csv', 'w', newline='') as file:
#     mywriter = csv.writer(file, delimiter=',')
#     mywriter.writerows(vectors_arr)

# **Prepare Train data for train**

## **Vectors**

**lenght of vectors**

In [25]:
vectors_len = [len(x) for x in vectors_arr]
vectors=np.array(vectors_len)
max_vector_len=max(vectors_len)

**Add padding to short comemnts**

In [26]:
#add padding vetor 0
padding_vec=[0.0]*100
padding_vec=np.array(padding_vec)
pad_embedded_array=[]
for vector in vectors_arr:
  if len(vector)<max_vector_len:
    padding=[padding_vec]*(max_vector_len-len(vector))
    vector=vector+padding
  pad_embedded_array.append(vector)

## **Labels**

In [ ]:
# from sklearn.preprocessing import LabelEncoder
# Le = LabelEncoder()
# y = Le.fit_transform(new_df['sentiment'])

In [27]:
l=train_df["label_id"]

# **Traingin LSTM**

**Import tensowerflow**

In [28]:
import tensorflow as tf
print(tf.__version__)

2.7.0


In [113]:
SEED = 42
AUTOTUNE = tf.data.AUTOTUNE

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [29]:
from keras.layers import Embedding
from keras.layers import Input
from keras.layers import TimeDistributed
from keras.layers import LSTM, Bidirectional
from keras.models import Model
from keras.layers.core import Dense, Dropout,Activation
import tqdm
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import sequence
from tensorflow.python.keras.layers import Input
from keras.models import Sequential
# from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

**Create Model**

In [30]:
nb_samples = len(padding_vec)#55000
time_steps=max_vector_len#74
input_dim=100 #100
input_shape=(time_steps,input_dim)
n_epoch=6
batch_size=32

In [31]:
# create the model
# embedding_weights = np.zeros((n_symbols, vocab_dim))
lstm_model = Sequential()
# model.add(LSTM(3NumberOfLSTM, return_sequences=True,input_shape=(YourSequenceLenght, YourWord2VecLenght)))
lstm_model.add(LSTM(batch_size,return_sequences=True, input_shape=input_shape))
lstm_model.add(Dropout(0.2))
lstm_model.add(Dense(16, activation='softmax'))
lstm_model.add(Dense(8, activation='softmax'))
lstm_model.add(Dense(3, activation='softmax'))
lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
lstm_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 74, 32)            17024     
                                                                 
 dropout (Dropout)           (None, 74, 32)            0         
                                                                 
 dense (Dense)               (None, 74, 16)            528       
                                                                 
 dense_1 (Dense)             (None, 74, 8)             136       
                                                                 
 dense_2 (Dense)             (None, 74, 3)             27        
                                                                 
Total params: 17,715
Trainable params: 17,715
Non-trainable params: 0
_________________________________________________________________


Train model

In [33]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=200)
mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

In [ ]:
x_train=pad_embedded_array
y_train=l
print( "Train..." )# batch_size=32
history =lstm_model.fit(x_train, y_train, batch_size=batch_size, epochs=n_epoch,verbose=1,callbacks=[es, mc])
# history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,
#                     callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

Train...


# **TEST**

**Prepare test data**

In [132]:
# test_token_df['tokenized_sents'] = test_token_df.apply(lambda row: fa_nlp(row['comment']).sentences.words.text, axis=1)
test_tokenized_array=[]
for i in range(0,len(dev_df)):
  comment=dev_df.iloc[i]["comment"]
  tokenized_comment = fa_nlp(comment)
  tokens_array=[]
  for i,sent in enumerate(tokenized_comment.sentences):
    for word in sent.words:
      tokens_array.append(word.text)
  test_tokenized_array.append(tokens_array)
#word.text, word.lemma, word.pos, word.head, word.deprel))

/usr/local/lib/python3.7/dist-packages/stanza/models/common/beam.py:86: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  prevK = bestScoresId // numWords


In [147]:
#save tokens to array
import csv
with open('test_tokens_array.csv', 'w', newline='') as file:
    mywriter = csv.writer(file, delimiter=',')
    mywriter.writerows(test_tokenized_array)

In [148]:
#read test array csv file
tokentest_tokenized_arrays_array=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/tokens_array.csv",sep='\t',header=None)

In [151]:
for i in test_tokenized_array[:3]:
  print((i))

['همه', 'ساندویچ\u200cها', 'سرد', 'بود', 'متاسفانه', 'در', 'صورتی', 'که', 'فاصله', 'تا', 'رستوران', 'کمتر', 'از', '۵', 'دقیقه', 'است', '.']
['تشکر', '،', 'عالی\u200cتر', 'از', 'همیشه']
['مرغ', 'و', 'کدو', 'رو', 'دوس', 'نداشتم', 'ولی', 'کلا', 'ساندویچهای', 'خوشمزه\u200cای', 'داره']


In [152]:
test_vectors_arr=[]
for sent in test_tokenized_array:
  vec_vocab=[model[word] for word in sent if word in vocabulary]# if model[word]
  test_vectors_arr.append(vec_vocab)

In [153]:
vectors_test_len = [len(x) for x in test_vectors_arr]
vectors_test_len=np.array(vectors_test_len)
max_vector_test_len=max(vectors_test_len)
#add padding vetor 0
# padding_vec=[0.0]*100
# padding_vec=np.array(padding_vec)
pad_embedded_test_array=[]
for vector in test_vectors_arr:
  if len(vector)<max_vector_test_len:
    padding=[padding_vec]*(max_vector_test_len-len(vector))
    vector=vector+padding
  pad_embedded_test_array.append(vector)

In [154]:
x_test=pad_embedded_test_array
y_test=dev_df["label_id"]

In [156]:
for i in x_test:
  if len(i)!=206:
    print(len(i))

In [ ]:
# evaluate the model
scores = lstm_model.evaluate(x_test, y_test, verbose=0)

Test Model

In [ ]:
#for test
# import pickle
# import sys
# import os
# from keras.preprocessing.sequence import pad_sequences
# from keras.utils.np_utils import to_categorical
# from keras.layers import TimeDistributed

# embedding_layer = Embedding(len(word2int) + 1,EMBEDDING_DIM,weights=[embedding_matrix],input_length=30,trainable=True)                         
# sequence_input = Input(shape=(30,), dtype='int32')
# embedded_sequences = embedding_layer(sequence_input)
# l_lstm = Bidirectional(LSTM(69, return_sequences=True))(embedded_sequences)

In [ ]:
# lstm_model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
# lstm_model.add(Dense(1, activation='sigmoid'))
# lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# print(model.summary())
# model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=64)